In [ ]:
##Cybase_Crawbing
import openpyxl as openpyxl
import requests
from lxml import etree
from urllib3 import disable_warnings
from urllib3.exceptions import InsecureRequestWarning

wb = openpyxl.Workbook()
sheet1 = wb.active
sheet1.append(['cid', 'name', 'seq', 'classc', 'mass', 'organism', 'family', 'link', 'reference', 'structure'])
sheet2 = wb.create_sheet('similar')
sheet2.title = 'similar'
sheet2.append(['similar'])
sheet1.sheet_properties.tabColor = '000000'
disable_warnings(InsecureRequestWarning)
url_main = 'https://www.cybase.org.au/index.php?table=protein&page=list&text=&textfield=All+fields&cy_mass' \
           '=&mass_range=1&mass_type=Average+Mass&cy_sequence=&sequence_type=subsequence&e_limit=10&Class%5B%5D=ALL' \
           '&ProteinType%5B%5D=ALL&Organism_search%5B%5D=ALL&Plant_family%5B%5D=ALL&Cyclic%5B%5D=Yes&Assay%5B%5D=ALL' \
           '&fields%5B%5D=ID&fields%5B%5D=Name&fields%5B%5D=Sequence&fields%5B%5D=Class&fields%5B%5D=Technique&fields' \
           '%5B%5D=Average+Mass&fields%5B%5D=Parent&fields%5B%5D=Domains&fields%5B%5D=Nucleic+acids&fields%5B%5D' \
           '=Organism&fields%5B%5D=Plant_family&fields%5B%5D=References&fields%5B%5D=Assay&fields%5B%5D=Structure' \
           '&fields%5B%5D=Links'
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.54 "
}
response = requests.get(url=url_main, headers=headers, verify=False)
response.encoding = response.apparent_encoding
tree = etree.HTML(response.text)
tr_list = tree.xpath('//*[@id="datatable"]/tr')
for tr in tr_list:
    cid, name, seq, classc, mass, organism, family, link, reference, structure = '', '', '', '', '', '', '', '', '', ''
    cid = ''.join(tr.xpath('./td[2]/text()'))
    if cid != '':
        classc = ''.join(tr.xpath('./td[5]/text()'))
        organism_list = tr.xpath('./td[11]/i')
        organism = '##'.join(tr.xpath('./td[11]//text()'))
        if 'Organism' in tr.xpath('./td[10]/b'):
            organism_list = tr.xpath('./td[10]/i')
            organism = '##'.join(organism_list.xpath('.//text()'))
        name = ''.join(tr.xpath('./td[3]/a/text()'))
        classc = ''.join(tr.xpath('./td[5]/text()'))
        mass = ''.join(tr.xpath('./td[7]/text()'))
        seq = ''.join(tr.xpath('./td[4]/text()'))

        family = ''.join(tr.xpath('./td[12]//text()'))
        link = '##'.join(tr.xpath('./td[16]//text()')).replace(" ##", ":")
        ref = tr.xpath('./td[13]/a')
        for re in ref:
            reference += ''.join(re.xpath('./@href')).replace('http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?db=pubmed&cmd=Retrieve&dopt=AbstractPlus&list_uids=', '').replace('&query_hl=1&itool=pubmed_docsum', '') + '##'
        struct = tr.xpath('./td[15]/a')
        for re in struct:
            structure += ''.join(re.xpath('./@href')).replace('?page=card&table=structure&id=', '') + "##"
        xx_info = [cid, name, seq, classc, mass, organism, family, link, reference.rstrip("##"), structure.rstrip("##")]
        print(xx_info)
        sheet1.append(xx_info)
wb.save('cybase_Info.xlsx')